<a href="https://colab.research.google.com/github/LBY-L/MineColab/blob/master/MineColab_forked_by_LBY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from IPython.core.display import display, HTML
display(HTML('<p><img style="display: block; margin-left: auto; margin-right: auto;" src="https://github.com/LBY-L/MineColab/blob/master/Logo.png?raw=true" alt="" width="170" height="136" /></p><h1 style="text-align: center;"><span style="color: #99ccff;">MineColab</span></h1><hr /><h2 style="text-align: center;"><span style="color: #ccffcc;">Forked By LBY_L</span></h2><h2 style="text-align: center;"><span style="color: #99cc00;">Run Minecraft Server On Google Colab</span></h2>'))

In [1]:
#@title <p><img src="https://github.com/LBY-L/MineColab/blob/master/grass.png?raw=true" alt="grass.png" width="90" height="90" /></p><hr /><h2> { display-mode: "form" }
#@markdown #### **You can chose betwen a suffix of your server folder.**
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')
suffix = "au" #@param ["colab", "cts", "combat", "velocity", "flypvp", "adrip", "rizecookey", "lby", "pvp", "us", "eu", "ap", "au", "sa", "jp", "in"]
!mkdir "/content/drive/MyDrive/Minecraft-server-$suffix"

#@markdown Your folder must be look like this `Minecraft-server-suffix`

#@markdown #### **Run the cell to create folder**

Mounted at /content/drive


In [1]:
#@title <p><img src="https://github.com/LBY-L/MineColab/blob/master/Logo.png?raw=true" width="96" height="77" /></p><hr /><h2>Click here to run your server. Before you need follow the instructions</h2> { display-mode: "form" }



JarName = "server.jar" #@param {type:"string"}
Paper = False #@param {type:"boolean"}
#@markdown #### **You can select between [Ngrok](https://ngrok.com/) or [PlayIt](https://playit.gg/) if you will use Ngrok you need complete NgrokAuthToken and NgrokRegion.**
TunnelService = "playit" #@param ["ngrok", "playit"]
#@markdown #### **Here put your ngrok token, if you haven't it, get it [here](https://dashboard.ngrok.com/get-started/your-authtoken).**
NgrokAuthToken = "" #@param {type:"string"}
#@markdown #### **Here is the region where is will be hosted your minecraft.**
#@markdown * #### `us` - United States (Ohio)
#@markdown * #### `eu` - Europe (Frankfurt)
#@markdown * #### `ap` - Asia/Pacific (Singapore)
#@markdown * #### `au` - Australia (Sydney)
#@markdown * #### `sa` - South America (Sao Paulo)
#@markdown * #### `jp` - Japan (Tokyo)
#@markdown * #### `in` - India (Mumbai)
NgrokRegion = "None" #@param ["us", "eu", "ap", "au", "sa", "jp", "in", "None"]

from google.colab import drive
from IPython.core.display import display, HTML, clear_output
import json
import os
import re
import subprocess
import time

drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Minecraft-server-$suffix"
!echo "eula=true" > eula.txt

def updateBar(a):
  out.update(progress(a, 100))

def seCls(b):
  time.sleep(b)
  clear_output()

def console():
  display(HTML('<p><img style="display: block; margin-left: auto; margin-right: auto;" src="https://github.com/LBY-L/MineColab/blob/master/CONSOLE-12-12-2022.png?raw=true" alt="CONSOLE-12-12-2022.png" width="511" height="175" /></p>'))

seCls(0)

# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/MyDrive/Minecraft-server-$suffix"

seCls(0)

# Install Java Versions (8, 17)
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"; sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."; sudo apt-get install openjdk-8-jre-headless &>/dev/null && echo "Yay! Openjdk8 has been successfully installed." || echo "Failed to install OpenJdk8."

# Java arguments.
if Paper == True:
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = ""; memory_allocation = "-Xmx6144M -Xms6144M" # aiker's flags might negatively impact performance on non-paper servers.

seCls(0)

# Display html text of Starting server
display(HTML('<p><img style="display: block; margin-left: auto; margin-right: auto;" src="https://github.com/LBY-L/MineColab/blob/master/COLAB-26-11-2022.png?raw=true" alt="COLAB-26-11-2022.png" width="327" heig<p><img style="display: block; margin-left: auto; margin-right: auto;" src="https://github.com/LBY-L/MineColab/blob/master/Starting-Serever-12-12-2022.png?raw=true" alt="Starting-Serever-12-12-2022.png" width="335" height="91" /></p>'))
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

out = display(progress(0, 100), display_id=True)

# Server starting
subprocess.run(["python", "-c", "'while True:pass'"])
if TunnelService == "ngrok":
  !pip -q install pyngrok &>/dev/null

  updateBar(16.7)

  from pyngrok import conf, ngrok

  updateBar(33.4)

  # Ask for the ngrok authtoken
  !ngrok authtoken $NgrokAuthToken &>/dev/null # login to ngrok

  updateBar(50.1)

  # Sets default ngrok region
  conf.get_default().region = NgrokRegion  # Change this to whichever region you want

  updateBar(66.8)

  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')

  updateBar(83.5)

  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

  updateBar(100.2)
  seCls(3)

  console()
  !java $memory_allocation $server_flags -jar $JarName nogui

elif TunnelService == "playit":
  !curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add - &>/dev/null

  updateBar(33.4)

  !sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list &>/dev/null

  updateBar(66.8)

  !sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "PlayIt installed" || echo "Failed to install PlayIt"

  updateBar(100.2)
  seCls(3)

  console()

  !playit & java $memory_allocation $server_flags -jar $JarName nogui


ValueError: ignored